In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, KFold
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('Data_Set/Perovskite_database_content_all_data.csv')

In [ ]:
data.shape

In [4]:
data

,Ref_ID,Ref_ID_temp,Ref_name_of_person_entering_the_data,Ref_data_entered_by_author,Ref_DOI_number,Ref_lead_author,Ref_publication_date,Ref_journal,Ref_part_of_initial_dataset,Ref_original_filename_data_upload,...,Outdoor_PCE_Tse80,Outdoor_PCE_after_1000_h,Outdoor_power_generated,Outdoor_link_raw_data_for_outdoor_trace,Outdoor_detaild_weather_data_available,Outdoor_link_detailed_weather_data,Outdoor_spectral_data_available,Outdoor_link_spectral_data,Outdoor_irradiance_measured,Outdoor_link_irradiance_data
0,1,1,Adam Hultqvist,False,10.1021/jp5126624,Sabba et al.,2015-01-06,The Journal of Physical Chemistry C,True,Historic dataset on 2020 11 22_v7.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
1,2,2,Adam Hultqvist,False,10.1021/jp5126624,Sabba et al.,2015-01-06,The Journal of Physical Chemistry C,True,Historic dataset on 2020 11 22_v7.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
2,3,3,Adam Hultqvist,False,10.1021/jp5126624,Sabba et al.,2015-01-06,The Journal of Physical Chemistry C,True,Historic dataset on 2020 11 22_v7.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
3,4,4,Adam Hultqvist,False,10.1021/jp5126624,Sabba et al.,2015-01-06,The Journal of Physical Chemistry C,True,Historic dataset on 2020 11 22_v7.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
4,5,5,Adam Hultqvist,False,10.1021/jp5126624,Sabba et al.,2015-01-06,The Journal of Physical Chemistry C,True,Historic dataset on 2020 11 22_v7.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42492,42497,37,Timothy Siegler,True,10.1021/acsaem.9b01298,Siegler et. al.,2019-07-12,-,False,2022-04-05 _v1.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
42493,42485,25,Timothy Siegler,True,10.1021/acsaem.9b01298,Siegler et. al.,2019-07-12,-,False,2022-04-05 _v1.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
42494,42488,28,Timothy Siegler,True,10.1021/acsaem.9b01298,Siegler et. al.,2019-07-12,-,False,2022-04-05 _v1.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN
42495,42491,31,Timothy Siegler,True,10.1021/acsaem.9b01298,Siegler et. al.,2019-07-12,-,False,2022-04-05 _v1.xlsx,...,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN


In [ ]:
data['Perovskite_thickness']

In [6]:
def replace(x):
    if x in ['Unknown','none','unknown','None']:
        return np.nan
    elif 'Unknown' in str(x):
        return np.nan
    elif 'none' in str(x):
        return np.nan
    elif 'None' in str(x):
        return np.nan
    elif 'nan' in str(x):
        return np.nan
    else:
        return x

In [7]:
data=data.applymap(lambda x :replace(x))

In [ ]:
data.shape

In [9]:
data=data.applymap(lambda x: x.replace(' ','') if isinstance(x,str) else x)

In [10]:
data.drop_duplicates(inplace=True)

In [11]:
data.dropna(axis='columns', how ='all',inplace=True)

In [12]:
data.dropna(axis='rows', how ='all',inplace=True)

In [ ]:
data.shape

In [14]:
data.dropna(subset=['Perovskite_composition_a_ions',
 'Perovskite_composition_a_ions_coefficients',
 'Perovskite_composition_b_ions',
 'Perovskite_composition_b_ions_coefficients',
 'Perovskite_composition_c_ions',
 'Perovskite_composition_c_ions_coefficients','Perovskite_dimension_3D','Perovskite_thickness','Perovskite_band_gap'],inplace=True)

In [ ]:
data.shape

In [16]:
perovskite_comp=['Perovskite_composition_a_ions',
 'Perovskite_composition_a_ions_coefficients',
 'Perovskite_composition_b_ions',
 'Perovskite_composition_b_ions_coefficients',
 'Perovskite_composition_c_ions',
 'Perovskite_composition_c_ions_coefficients','Perovskite_dimension_3D','Perovskite_thickness','Perovskite_band_gap']

In [ ]:
perovskite_comp

In [18]:
perovskite_data=data[perovskite_comp]

In [ ]:
perovskite_data.shape

In [20]:
perovskite_data=perovskite_data.dropna()

In [21]:
perovskite_data['Layered']=perovskite_data['Perovskite_composition_a_ions'].apply(lambda x: True if len(str(x).split('|'))>1 else False)

In [22]:
perovskite_data=perovskite_data[~perovskite_data['Layered']]

In [23]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_dimension_3D']]

In [24]:
same_dict={'Blowinghotair':'Blow-drying',
           'Comersial':'Commercial',
           'Electrohemicalanodization':'Electrochemicalanodization',
           'Electropolymerisation':'Electropolymerization',
           'Evaoration': 'Evaporation',
           'Gasreaction':'Gas-reaction',
           'Hydorthermal':'Hydorthermal',
           'Hydrolys':'Hydrolysis',
           'Inkjetprinting' :'Inkjet-Printed',
           'RFMagnetronSputtering': 'RFMagnetronsputtering',
           'RFmagnetronsputtering': 'RFMagnetronsputtering',
           'SILARmethod':'SILAR',
           'Screenprintnig':'Screenprinting',
           'Sputering':'Sputtering',
           'Ultrasonicspraypyrolysis':'Ultrasonicspray'}

In [25]:
perovskite_data.drop_duplicates(inplace=True)
perovskite_data.dropna(inplace=True)

In [26]:
a_ions=perovskite_data['Perovskite_composition_a_ions'].str.split(';').str.join(' ').str.split(expand=True).apply(pd.Series.value_counts).sum(axis=1)

In [27]:
a_ions=a_ions[a_ions>200].index.tolist()

In [28]:
b_ions=perovskite_data['Perovskite_composition_b_ions'].str.split(';').str.join(' ').str.split(expand=True).apply(pd.Series.value_counts).sum(axis=1)

In [29]:
b_ions=b_ions[b_ions>=100].index.tolist()

In [30]:
c_ions=perovskite_data['Perovskite_composition_c_ions'].str.split(';').str.join(' ').str.split(expand=True).apply(pd.Series.value_counts).sum(axis=1)

In [31]:
c_ions=c_ions[c_ions>200].index.tolist()

In [32]:
ions_token=list(set(a_ions+b_ions+c_ions))

In [33]:
perovskite_data.drop_duplicates(subset=['Perovskite_composition_a_ions','Perovskite_composition_b_ions','Perovskite_composition_c_ions','Perovskite_thickness'],inplace=True)

In [34]:
perovskite_data['Perovskite_thickness']=perovskite_data['Perovskite_thickness'].apply(lambda x: [float(x)/1000])

In [35]:
features=['Cs', 'FA', 'MA','Pb', 'Sn','Br', 'I','Thickness']

In [36]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_composition_a_ions'].apply(lambda x : all([True if y in ions_token else False for y in x.split(';') ]))]      

In [37]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_composition_b_ions'].apply(lambda x : all([True if y in ions_token else False for y in x.split(';') ]))]        

In [38]:
perovskite_data=perovskite_data[perovskite_data['Perovskite_composition_c_ions'].apply(lambda x : all([True if y in ions_token else False for y in x.split(';') ]))]        

In [39]:
def ions_one_hot(x,c,tokens):
    if 'x' in c:
        return np.nan
    temp_dict={a:b for (a,b) in zip(x.split(';'),c.split(';'))}
    return [temp_dict[x] if x in temp_dict.keys() else 0 for x in tokens]

In [40]:
perovskite_data['Perovskite_composition_a_ions']=perovskite_data[['Perovskite_composition_a_ions','Perovskite_composition_a_ions_coefficients']].apply(lambda x :ions_one_hot(*x,a_ions),axis=1)

In [41]:
perovskite_data['Perovskite_composition_b_ions']=perovskite_data[['Perovskite_composition_b_ions','Perovskite_composition_b_ions_coefficients']].apply(lambda x :ions_one_hot(*x,b_ions),axis=1)

In [42]:
perovskite_data['Perovskite_composition_c_ions']=perovskite_data[['Perovskite_composition_c_ions','Perovskite_composition_c_ions_coefficients']].apply(lambda x :ions_one_hot(*x,c_ions),axis=1)

In [43]:
perovskite_data

,Perovskite_composition_a_ions,Perovskite_composition_a_ions_coefficients,Perovskite_composition_b_ions,Perovskite_composition_b_ions_coefficients,Perovskite_composition_c_ions,Perovskite_composition_c_ions_coefficients,Perovskite_dimension_3D,Perovskite_thickness,Perovskite_band_gap,Layered
15,"[0, 0, 1]",1,"[1, 0]",1,"[0, 3]",3,True,[0.4],1.6,False
53,"[0, 0, 1]",1,"[1, 0]",1,"[0, 3]",3,True,[0.35],1.6,False
71,"[0, 0, 1]",1,"[1, 0]",1,"[0, 3]",3,True,[0.45],1.6,False
79,"[0, 0, 1]",1,"[1, 0]",1,"[0, 3]",3,True,[0.2],1.6,False
129,"[0, 1, 0]",1,"[1, 0]",1,"[0, 3]",3,True,[0.33],1.48,False
...,...,...,...,...,...,...,...,...,...,...
42308,"[0.08, 0.92, 0]",0.08;0.92,"[1.0, 0]",1.0,"[0, 3.0]",3.0,True,[0.7],1.51,False
42321,"[0.1, 0.75, 0.15]",0.1;0.75;0.15,"[1, 0]",1,"[0.5, 2.5]",0.5;2.5,True,[0.37],1.63,False
42384,"[0.05, 0.55, 0.4]",0.05;0.55;0.4,"[1, 0]",1,"[2.88, 0.12]",2.88;0.12,True,[0.3],1.56,False
42388,"[0.17, 0.83, 0]",0.17;0.83,"[1, 0]",1,"[2.59, 0.51]",2.59;0.51,True,[0.42],1.64,False


In [44]:
perovskite_data.dropna(inplace=True)

In [45]:
for cols in ['Perovskite_composition_a_ions',
       'Perovskite_composition_b_ions',
       'Perovskite_composition_c_ions']:
    maximum=perovskite_data[cols].map(len).max()
    perovskite_data[cols]=perovskite_data[cols].apply(lambda x: x+[0]*(maximum-len(x)))

In [46]:
perovskite_data['X_train']=perovskite_data[['Perovskite_composition_a_ions',
       'Perovskite_composition_b_ions',
       'Perovskite_composition_c_ions']].sum(axis=1)

In [ ]:
perovskite_data

In [48]:
train=np.asanyarray(perovskite_data['X_train'].tolist(),dtype=float)

In [49]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [50]:
label=np.asanyarray(perovskite_data['Perovskite_band_gap'].tolist(),dtype=float)

In [51]:
from sklearn.utils import shuffle

train , label = shuffle(train,label, random_state=12)

In [52]:
# save to csv file
from numpy import savetxt
savetxt('Data_Set/train.csv', train, delimiter=',')
savetxt('Data_Set/label.csv', label, delimiter=',')

In [53]:
from numpy import loadtxt
# load array
train = loadtxt('Data_Set/train.csv', delimiter=',')
label = loadtxt('Data_Set/label.csv', delimiter=',')

In [ ]:
train.shape

In [ ]:
import matplotlib.pyplot as plt

# Create the data
data = [2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 10]

# Plot the histogram
plt.hist(train, bins=8)

# Add a title and labels
plt.title("Histogram of Data")
plt.xlabel("Value")
plt.ylabel("Frequency")

# Show the plot
plt.show()